In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

In [43]:
current_date = datetime.now()
current_date = current_date.replace(second=0, microsecond=0)

In [44]:
# Corresponding columns for date, longitude, latitude, quantity(number), weight(kg)
columns_output = ["date", "latitude", "longitude", "quantity", "weight" ]
output_path = "..\database.csv"

In [45]:
file_path = "MDT Data_April 2020.csv"

columns_input = ["dt", "longitude", "latitude", "quantity", "country"]

df = pd.read_csv(file_path, header=0, usecols=columns_input)

df = df[df["country"] == "NO"]
df["weight"] = np.nan

df["date"] = pd.to_datetime(df["dt"])
df["quantity"] = pd.to_numeric(df["quantity"])

df[columns_output].to_csv(output_path)

In [46]:
from pyproj import Proj
from shapely import wkt

def convert_wkt_to_gps(df):
    # Create Proj object for coversion from EPSG:32633
    myProj = Proj("+proj=utm +zone=33 +datum=WGS84 +units=m +no_defs")

    # Covert wkt string to shapely point object
    df["geom_wkt"] = df["geom_wkt"].apply(wkt.loads)

    longitude_points = df["geom_wkt"].apply(lambda point: point.x)
    latitude_points = df["geom_wkt"].apply(lambda point: point.y)

    df["longitude"], df["latitude"] = myProj(longitude_points.values, latitude_points.values, inverse=True)
    
    return df

In [47]:
file_path = "Rydde - Aksjoner med funn _Ny protokoll_.csv"

columns_input = ["enddate", "geom_wkt", "kg_of_collected_waste"]

df = pd.read_csv(file_path, header=0, usecols=columns_input)

df = df.rename(columns={"kg_of_collected_waste": "weight"})

df["date"] = pd.to_datetime(df["enddate"], format="%d.%m.%Y %H:%M")

df.loc[df["date"] > current_date, "date"] = current_date

df["quantity"] = np.nan

convert_wkt_to_gps(df)

df[columns_output].to_csv(output_path, mode="a", header=False)

In [48]:
file_path = "Kopi av Rydde - Aksjoner med funn _Gammel protokoll_.csv"

columns_input = ["enddate", "geom_wkt", "kg_of_collected_waste"]

df = pd.read_csv(file_path, header=0, usecols=columns_input)

df = df.rename(columns={"kg_of_collected_waste": "weight"})

df["date"] = pd.to_datetime(df["enddate"], format="%d.%m.%Y %H:%M")

df.loc[df["date"] > current_date, "date"] = current_date

df["quantity"] = np.nan

convert_wkt_to_gps(df)

df[columns_output].to_csv(output_path, mode="a", header=False)

In [49]:
file_path = "SALT MAP+Kvantesprang raw data COMPLETE.csv"

columns_input = ["Sampling date", "latitude start", "longitude start", "TOTAL LITTER (#)", "TOTAL LITTER (g)"]

df = pd.read_csv(file_path, header=0, usecols=columns_input)

df["date"] = pd.to_datetime(df["Sampling date"])

df["weight"] = pd.to_numeric(df["TOTAL LITTER (g)"].str.replace(",",""))/1000
df["quantity"] = pd.to_numeric(df["TOTAL LITTER (#)"].str.replace(",",""))

df = df.rename(columns={"latitude start": "latitude"})
df = df.rename(columns={"longitude start": "longitude"})

df[columns_output].to_csv(output_path, mode="a", header=False)